In [1]:
from sklearn.tree import export_text
from sklearn.datasets import load_files
from sklearn import preprocessing
from sklearn.utils import shuffle
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.tree import DecisionTreeClassifier
import itertools
import time
from sklearn.neighbors import KNeighborsClassifier
import mlrose_hiive
from sklearn.metrics import accuracy_score
from sklearn import metrics


In [2]:
cars = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data", header = None)
#Adding the column names
cars_col_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
cars.columns = cars_col_names

import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

cars['class'] = cars['class'].apply(lambda x: 0 if x == 'unacc' else 1)

X = cars.drop(['class'], axis=1)


y = cars['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

Random Hill Climb

In [21]:
st_time = time.time()
grid_search_parameters = {
    'max_iters': [1000],                          # nn params
    'learning_rate': [1e-2],                       # nn params
    'activation': [mlrose_hiive.relu],            # nn params
    'restarts': [0],                             # rhc params
}

nnr = mlrose_hiive.NNGSRunner(
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    experiment_name='name',
    algorithm=mlrose_hiive.algorithms.rhc.random_hill_climb,
    grid_search_parameters=grid_search_parameters,
    iteration_list=range(1, 2001),
    hidden_layer_sizes=[[60]],
    bias=True,
    early_stopping=False,
    clip_max=5,
    max_attempts=2000,
    n_jobs=5,
    seed=1,
    output_directory=None
)

start = time.time()
run_stats_df, crvs, cv_results_df, grid_search_cv = nnr.run()
print("Random Hill Climbing Numbers:")

y_train_pred = grid_search_cv.predict(X_train)
y_train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Acc: {y_train_accuracy}")

y_test_pred = grid_search_cv.predict(X_test)
y_test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Acc: {y_test_accuracy}")

print(f"Function Evaluations: {crvs['FEvals'][1000]}")
print(f"Wall clock time: {time.time() - start}")

print("Raw data: ")
print(crvs)

plt.xlabel("Iteration")
plt.ylabel("Fitness Score")
plt.title(f"Fitness Score versus Iteration for RHC")
plt.plot(range(1,len(crvs['Fitness'])+1), crvs['Fitness'], label="RHC")
plt.savefig(f"plots/NN-RHC.png")
plt.close()


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Random Hill Climbing Numbers:
Training Acc: 0.8002894356005789
Test Acc: 0.7630057803468208
Function Evaluations: 1365.0
Wall clock time: 50.91121745109558
Raw data: 
      Iteration       Time   Fitness  FEvals  max_iters  learning_rate  \
0             0   0.005652  2.479490     1.0       1000           0.01   
1             1   0.020461  2.479490     2.0       1000           0.01   
2             2   0.036053  2.479490     3.0       1000           0.01   
3             3   0.048540  2.479490     4.0       1000           0.01   
4             4   0.061686  2.479490     5.0       1000           0.01   
...         ...        ...       ...     ...        ...            ...   
996         996  19.021087  0.428292  1360.0       1000           0.01   
997         997  19.043867  0.428292  1361.0       1000           0.01   
998         998  19.067950  0.427164  1363.0       1000           0.01   
999         999  19.088615  0.4271

AttributeError: 'NNGSRunner' object has no attribute 'plot_iteration_vs_fitness'

Simulated Annealing

In [14]:
grid_search_parameters = {
    'max_iters': [2000],
    'learning_rate': [1e-2],
    'activation': [mlrose_hiive.relu],
    'restarts': [0],
} 

nnr_sa = mlrose_hiive.NNGSRunner(
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    experiment_name='name',
    algorithm=mlrose_hiive.algorithms.sa.simulated_annealing,
    grid_search_parameters=grid_search_parameters,
    iteration_list=range(1, 2001),
    hidden_layer_sizes=[[60]],
    bias=True,
    early_stopping=False,
    clip_max=5,
    max_attempts=1000,
    n_jobs=5,
    seed=1,
    output_directory=None
)

start = time.time()
run_stats_df, curves_df, cv_results_df, grid_search_cv = nnr_sa.run()

print("Simulated Annealing Numbers:")

y_train_pred = grid_search_cv.predict(X_train)
y_train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Acc: {y_train_accuracy}")

y_test_pred = grid_search_cv.predict(X_test)
y_test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Acc: {y_test_accuracy}")


print(f"Function Evaluations: {curves_df['FEvals'][2000]}")
print(f"Wall clock time: {time.time() - start}")

print("Raw data: ")
print(curves_df)

plt.xlabel("Iteration")
plt.ylabel("Fitness Score")
plt.title(f"Fitness Score versus Iteration for SA")
plt.plot(range(1,len(curves_df['Fitness'])+1), curves_df['Fitness'], label="RHC")
plt.savefig(f"plots/NN-SA.png")
# plt.show()
plt.close()

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Simulated Annealing Numbers:
Training Acc: 0.8154848046309696
Test Acc: 0.7976878612716763
Function Evaluations: 3516.0
Wall clock time: 107.22919297218323
Raw data: 
      Iteration       Time   Fitness  FEvals  max_iters  learning_rate  \
0             0   0.007386  2.494543     2.0       2000           0.01   
1             1   0.024596  2.494543     4.0       2000           0.01   
2             2   0.040514  2.502183     6.0       2000           0.01   
3             3   0.056738  2.503810     8.0       2000           0.01   
4             4   0.073257  2.420462    10.0       2000           0.01   
...         ...        ...       ...     ...        ...            ...   
1996       1996  47.408260  0.376943  3508.0       2000           0.01   
1997       1997  47.440909  0.377275  3510.0       2000           0.01   
1998       1998  47.473036  0.377256  3512.0       2000           0.01   
1999       1999  47.504269  0.3768

Genetic Algorithm

In [20]:
grid_search_parameters = {
    'max_iters': [2000],
    'learning_rate': [1e-2],
    'activation': [mlrose_hiive.relu],
    'restarts': [0],
} 

nnr_ga = mlrose_hiive.NNGSRunner(
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    experiment_name='name',
    algorithm=mlrose_hiive.algorithms.ga.genetic_alg,
    grid_search_parameters=grid_search_parameters,
    iteration_list=range(1, 2001),
    hidden_layer_sizes=[[60]],
    bias=True,
    early_stopping=False,
    clip_max=5,
    max_attempts=1000,
    n_jobs=5,
    seed=1,
    output_directory=None
)

start = time.time()
run_stats_df, curves_df, cv_results_df, grid_search_cv = nnr_ga.run()

print("Simulated Annealing Numbers:")

y_train_pred = grid_search_cv.predict(X_train)
y_train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Acc: {y_train_accuracy}")

y_test_pred = grid_search_cv.predict(X_test)
y_test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Acc: {y_test_accuracy}")


print(f"Function Evaluations: {curves_df['FEvals'][2000]}")
print(f"Wall clock time: {time.time() - start}")

print("Raw data: ")
print(curves_df)

plt.xlabel("Iteration")
plt.ylabel("Fitness Score")
plt.title(f"Fitness Score versus Iteration for GA")
plt.plot(range(1,len(curves_df['Fitness'])+1), curves_df['Fitness'], label="GA")
plt.savefig(f"plots/NN-GA.png")
plt.show()
plt.close()

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Simulated Annealing Numbers:
Training Acc: 0.8191027496382055
Test Acc: 0.8034682080924855
Function Evaluations: 402211.0
Wall clock time: 2521.1324462890625
Raw data: 
      Iteration         Time   Fitness    FEvals  max_iters  learning_rate  \
0             0     0.499697  6.969753     201.0       2000           0.01   
1             1     1.029312  6.969753     403.0       2000           0.01   
2             2     1.557786  6.969753     604.0       2000           0.01   
3             3     2.068082  6.969753     805.0       2000           0.01   
4             4     2.601198  6.337739    1007.0       2000           0.01   
...         ...          ...       ...       ...        ...            ...   
1996       1996  1119.686882  3.154644  401407.0       2000           0.01   
1997       1997  1120.250450  3.154644  401608.0       2000           0.01   
1998       1998  1120.820063  3.154644  401809.0       2000           